In [95]:
import zlib
  
s = b'\xba\xbe\xca\xfe'
# using zlib.crc32() method
t_with_def_seed0 = zlib.crc32(s)

#t_with_seed = zlib.crc32(s, 67)
  
# to check if they are signed or unsigned?
print(hex(t_with_def_seed0))
#print(t_with_seed)

def crc32mpeg2(buf, crc=0x0):
    
    for val in buf:
        crc ^= val << 24
        for _ in range(8):
            crc = crc << 1 if (crc & 0x80000000) == 0 else (crc << 1) ^ 0x104c11db7
    return crc

crc_out = crc32mpeg2(s)
print(hex(crc_out))

0xf75b4227
0x6272462c


In [3]:
import crcengine as crceng
params = crceng.CrcParams(0x864cfb, 24, 0xb704ce, reflect_in=False, reflect_out=False, xor_out=0)
crc_openpgp = crceng.create_from_params(params)
# this is equivalent to
crc_openpgp = crceng.create(params=params)
# invocation
result = crc_openpgp(b'123456789')
print(f'CRC=0x{result:08x}')

AttributeError: module 'crcengine' has no attribute 'CrcParams'

In [94]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=True, initCrc = 0x0, xorOut=0xffffffff)
print(hex(crc32(b'\xba\xbe\xca\xfe')))

0xf75b4227


In [30]:
init = 0xEDB88320

In [31]:
new = init << 1 or 0x1

In [29]:
print(hex(new))

0x1db710640


In [93]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=False, initCrc=0x0, xorOut=0xffffffff)
print(hex(crc32(b'\xba\xbe\xca\xfe')))

0x5a8964a8


In [12]:
init2 = 0xbabecafe
print(bin(init2))
10111010101111101100101011111110
#print(hex(b'01111111010100110111110101011101'))
new2 = 0x7f537d5d

0b10111010101111101100101011111110


In [42]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x1db710640, rev=True, initCrc=0xffffffff, xorOut=0xffffffff)
print(hex(crc32(b'0xbabecafe')))

0xffa2ac6e


In [24]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x1EDB88320, rev=False, initCrc=0xffffffff, xorOut=0xFFFFFFFF)
print(hex(crc32(b'0xbabecafe')))

ValueError: The degree of the polynomial must be 8, 16, 24, 32 or 64

In [50]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=False, initCrc=0xffffffff, xorOut=0xffffffff)
print(hex(crc32(b'0xd5d735f7')))

0x26f0072e


In [65]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=False, initCrc=0xffffffff, xorOut=0x0)
print(hex(crc32(b'0xbabecafe')))

0xa8a6a95f


In [70]:
import crcmod
import crcmod.predefined
s = 
crc32_func = crcmod.predefined.mkCrcFun('crc-32-mpeg')
print(hex(crc32_func('123456789')))


TypeError: Unicode-objects must be encoded before calculating a CRC

In [73]:
import crcmod.predefined

crc32 = crcmod.predefined.Crc('crc-32')
crc32.update(b'123456789')
print(hex(crc32.crcValue))
#print(hex(crc32_func('123456789')))

0xcbf43926


In [74]:
import crcmod.predefined
crc32 = crcmod.predefined.Crc('crc-32')
crc32.update(b'0xbabecafe')
print(hex(crc32.crcValue))

0x51a9d74


In [91]:
import crcmod.predefined
s = b'\xba\xbe\xca\xfe'
#h = s.decode("hex")
#h = "0xbabecafe"
b = bytearray(s)
crc32_func = crcmod.predefined.mkCrcFun('crc-32-mpeg')
print(hex(crc32_func((b))))

0xa5769b57


In [89]:
import crcmod.predefined
crc32 = crcmod.predefined.Crc('crc-32-mpeg')
#crc32.update(b'123456789')
crc32.update(b'\xba\xbe\xca\xfe')
print(hex(crc32.crcValue))

0xa5769b57
